In [1]:
# домашняя работа 

In [2]:
# 1. Теперь решаем задачу регрессии - предскажем цены на недвижимость. 
#    Использовать датасет 
#    https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data (train.csv)
# 2. Данных немного, поэтому необходимо использовать 10-fold кросс-валидацию 
#    для оценки качества моделей
# 3. Построить случайный лес, вывести важность признаков
# 4. Обучить стекинг как минимум 3х моделей, использовать 
#    хотя бы 1 линейную модель и 1 нелинейную
# 5. Для валидации модели 2-го уровня использовать 
#    отдельный hold-out датасет, как на занятии
# 6. Показать, что использование ансамблей моделей действительно 
#    улучшает качество (стекинг vs другие модели сравнивать на hold-out)
# 7. В качестве решения: Jupyter notebook с кодом, комментариями 
#    и графиками, ссылка на гитхаб


In [52]:
import pandas as pd
import numpy as np
import random

from sklearn.model_selection import train_test_split

In [53]:
import warnings
warnings.filterwarnings('ignore')

In [54]:
data = pd.read_csv('train.csv')
data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [55]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-n

In [56]:
# Заполнить средними значениеми
def put_mean_to_na_param(dataset, list_columns):
    for i in list_columns:
        dataset[i].fillna(dataset[i].mean(), inplace=True)

In [57]:
# Заполнить значением
def put_value_to_na_param(dataset, list_columns, value=0 ):
    for i in list_columns:
        dataset[i].fillna(value, inplace=True)

In [58]:
# Заполнить самым распространенным значением
def put_most_common_meaning_to_na_param(dataset, list_columns):
    for i in list_columns:
        dataset[i].fillna(dataset[i].value_counts()[0], inplace=True)

In [59]:
# Заполнить значения NaN значениеями из другой колонки
def put_param_from_col_src_to_na_param(dataset, col_dest, col_src): 
    dataset[col_dest]=dataset.apply(lambda x: x[col_src] if x[col_dest] is not None else x[col_dest], axis=1 )
            

In [60]:
# Закодировать nominal категориальную переменную с помощью "one hot encoding"  (используем реализацию из лекций)
def get_one_hot(df, cols):
    for each in cols:
        dummies = pd.get_dummies(df[each], prefix=each, drop_first=False)
        df = pd.concat([df, dummies], axis=1)
    return df 

# Закодировать ordered категориальную переменную используя числовой порядок
def get_ordered_col(dataset, col, dict_col_params):
    """
    В функцию передается датасет, имя колонки и словарь {'значение_параметра_в_колонке': числовое_значение_для_замены }
    """
    dataset[col + '_order'] = dataset.apply(lambda x: dict_col_params[x[col]],axis=1)
    return dataset

def get_ordered_cols(dataset, cols, dict_col_params):
    for col in cols:
        res = get_ordered_col(dataset, col, dict_col_params)
    return res.drop(columns=cols)     

dict_label_5_1 = {'Ex': 4,'Gd': 3,'TA': 2,'Fa': 1,'Po': 0}
dict_label_5_2 = {'Ex': 4,'Gd': 3,'TA': 2,'Fa': 1,'NA': 0}
dict_label_5_3 = {'Gd': 4, 'Av': 3, 'Mn': 2, 'No': 1, 'NA': 0} 
dict_label_6_1 = {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, 'NA': 0}
dict_label_7_1 = {'GLQ': 6, 'ALQ': 5, 'BLQ': 4, 'Rec': 3, 'LwQ': 2, 'Unf': 1, 'NA': 0} 


In [61]:
# функция подготовки данных
def get_data(dataset):
    put_value_to_na_param(dataset, ['MiscFeature','Alley','Fence','BsmtQual','PoolQC','BsmtCond',\
                                    'BsmtExposure','BsmtFinType1','BsmtFinType2','FireplaceQu',\
                                    'GarageType','GarageFinish','GarageQual','GarageCond'], 'NA' )
    
    put_value_to_na_param(dataset, ['MasVnrArea'], 0)
    put_value_to_na_param(dataset, ['MasVnrType'], 'none')                      
    put_mean_to_na_param(dataset, ['LotFrontage'])
    put_most_common_meaning_to_na_param(dataset, ['Electrical'])   
    put_param_from_col_src_to_na_param(dataset, 'GarageYrBlt', 'YearRemodAdd')     
    
    data_raw = dataset.copy()
    data_raw = get_one_hot(dataset, ['MiscFeature', 'Fence', 'Alley', 'Street', 'LotShape', 'LandContour', 'Utilities',\
                                     'LotConfig', 'LandSlope', 'BldgType', 'MasVnrType', 'Foundation', 'Heating', \
                                     'CentralAir', 'Electrical', 'PavedDrive', 'MSZoning','MSSubClass','Neighborhood','Condition1',\
                                     'Condition2','HouseStyle','RoofStyle','RoofMatl','Exterior1st',\
                                     'Exterior2nd','Functional','GarageType','GarageFinish',\
                                     'SaleType','SaleCondition'])
    
    data_raw = get_ordered_cols(data_raw, ['ExterQual', 'ExterCond', 'HeatingQC', 'KitchenQual'], dict_label_5_1)
    data_raw = get_ordered_cols(data_raw, ['PoolQC'], dict_label_5_2)
    data_raw = get_ordered_cols(data_raw, ['BsmtExposure'], dict_label_5_3)
    data_raw = get_ordered_cols(data_raw, ['BsmtQual', 'BsmtCond', 'FireplaceQu', 'GarageQual', 'GarageCond'] , dict_label_6_1)
    data_raw = get_ordered_cols(data_raw, ['BsmtFinType1', 'BsmtFinType2'] , dict_label_7_1) 
    
    y = data_raw['SalePrice'].copy()
    #data_raw= data_raw.drop(columns=['SaleCategory'])
    columns_to_drop = ['SalePrice', 'MSZoning','MSSubClass','Neighborhood','Condition1',\
                       'Condition2','HouseStyle','RoofStyle','RoofMatl','Exterior1st',\
                       'Exterior2nd','Functional','GarageType','GarageFinish',\
                       'SaleType','SaleCondition','MiscFeature', 'Fence', 'Alley',\
                       'Street', 'LotShape', 'LandContour', 'Utilities',\
                       'LotConfig', 'LandSlope', 'BldgType', 'MasVnrType', \
                       'Foundation', 'Heating', 'CentralAir', 'Electrical', 'PavedDrive']
    
    data_raw = data_raw.drop(columns=columns_to_drop)
    
        
    
    return data_raw, y
    

In [62]:
X_full, y_full = get_data(data)

In [63]:
X_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Columns: 263 entries, Id to BsmtFinType2_order
dtypes: float64(2), int64(47), uint8(214)
memory usage: 864.1 KB


In [64]:
X_full.head()

,Id,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,KitchenQual_order,PoolQC_order,BsmtExposure_order,BsmtQual_order,BsmtCond_order,FireplaceQu_order,GarageQual_order,GarageCond_order,BsmtFinType1_order,BsmtFinType2_order
0,1,65.0,8450,7,5,2003,2003,196.0,706,0,...,3,0,1,4,3,0,3,3,6,1
1,2,80.0,9600,6,8,1976,1976,0.0,978,0,...,2,0,4,4,3,3,3,3,5,1
2,3,68.0,11250,7,5,2001,2002,162.0,486,0,...,3,0,2,4,3,3,3,3,6,1
3,4,60.0,9550,7,5,1915,1970,0.0,216,0,...,3,0,1,3,4,4,3,3,5,1
4,5,84.0,14260,8,5,2000,2000,350.0,655,0,...,3,0,3,4,3,3,3,3,6,1


In [65]:
# X и y общая выборка для моделей 1 уровня 
# X_out,y_out - выборка для проверки результата

X, X_out, y, y_out = train_test_split(X_full.drop(columns=['Id']), y_full.values, test_size=0.1 ,random_state=42)

In [66]:
from sklearn.preprocessing import StandardScaler

In [67]:
sc = StandardScaler().fit(X)
X_keys = X.columns
X = sc.transform(X)
X_out = sc.transform(X_out)
X = pd.DataFrame(X, columns=X_keys)
X_out = pd.DataFrame(X_out, columns=X_keys)

In [68]:
# 3. Построить случайный лес, вывести важность признаков

from sklearn.ensemble import RandomForestRegressor

In [69]:
clf = RandomForestRegressor()
clf.fit(X,y)

feature_importances= {}
for col, val in zip(X.columns, clf.feature_importances_):
    feature_importances[col] = val
    
sorted_x = sorted(feature_importances.items(), key=lambda kv: kv[1], reverse=True)
sorted_x[:20]

[('OverallQual', 0.6026800496364005),
 ('GrLivArea', 0.10607128726816824),
 ('TotalBsmtSF', 0.046399807530782634),
 ('1stFlrSF', 0.02221884775987217),
 ('BsmtFinSF1', 0.021290374181854203),
 ('TotRmsAbvGrd', 0.016333746849365633),
 ('LotArea', 0.014811388722135879),
 ('GarageArea', 0.012345969293026789),
 ('2ndFlrSF', 0.011563624998739221),
 ('GarageCars', 0.00938493833529991),
 ('YearBuilt', 0.008432166657898765),
 ('BsmtQual_order', 0.008314588500461888),
 ('LotFrontage', 0.007012824197271251),
 ('FireplaceQu_order', 0.00645966244095948),
 ('OpenPorchSF', 0.006025605510361398),
 ('BsmtUnfSF', 0.005407182580186842),
 ('KitchenQual_order', 0.004734840745206518),
 ('BedroomAbvGr', 0.004375571074768719),
 ('PoolQC_order', 0.004355709083951524),
 ('WoodDeckSF', 0.004122498311660691)]

In [70]:
# 4. Обучить стекинг как минимум 3х моделей, использовать 
#    хотя бы 1 линейную модель и 1 нелинейную
# 5. Для валидации модели 2-го уровня использовать 
#    отдельный hold-out датасет, как на занятии

In [71]:
# Для стекинга используем следующие типы моделей
# модели первого уровня
# 1. SGDRegressor
# 2. Ridge
# 3. KNeighborsRegressor
# 4. LinearSVR
# 5. DecisionTreeRegressor
# 6. Lasso
# модель второго уровня
# 1. LinearRegression

# Из моделей первого уровня отберем те которые дают более менее адекватный результат

In [72]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.svm import LinearSVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Lasso
from sklearn.linear_model import SGDRegressor

from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

In [195]:
def make_stacking(X, y, dict_model, meta_model, param_model_GS, param_meta_model):
    # получаем длину фолда
    len_fold = (len(X) // len(dict_model.keys()))+1
    kf = KFold(n_splits=len(dict_model.keys()), shuffle=True)
    result_fold = {}
    i = 0
    for train_index, test_index in kf.split(X):
        full_test_index_l = list(test_index)
        train_index_l =  list(train_index)
        # для того что бы данные для мета модели были полные
        # выравниваем длину фолдов
        # переносим избыток индексов из train в test
        if(len_fold > len(test_index)):
            # получим номера индексов в train  для переноса в test   
            add_num = random.sample(range(len(train_index)), len_fold > len(test_index))
            # получим индексы из train
            list_change_elem = train_index[add_num]
            # пройдем по списку индексов 
            for l in list_change_elem:
                print(add_num, l)
                # удалим индекс из train
                train_index_l.remove(l)
                #  добавим индекс в test
                full_test_index_l = full_test_index_l + list([l,])
        # -----------------------        
        # формируем выборки данных для фолдов
        k = list(dict_model.keys())[i]
        result_fold[k] = {'X_train': X.iloc[train_index_l], 'X_test': X.iloc[full_test_index_l],
                          'y_train': y[train_index_l], 'y_test': y[full_test_index_l]}
        # ------------------------
        i+=1
    result_model ={}
    res = None
    # проходим по списку моделей 1 уровня 
    for j in dict_model.keys():
        # обучаем 
        dict_model[j] = GridSearchCV(dict_model[j], param_model_GS[j], cv=10).fit(result_fold[j]['X_train'], result_fold[j]['y_train']).best_estimator_
        # предсказания помещаем в словарь с данными для мета модели
        result_model[j] = dict_model[j].predict(result_fold[j]['X_test'])
        res = result_model[j] if type(res) == type(None) else res+result_model[j]
    print((res//len(dict_model.keys()))-result_fold[j]['y_test']) 
    df_result = pd.DataFrame.from_dict(result_model)
    # обучаем мета модель
    meta_model =  GridSearchCV(meta_model, param_meta_model, cv=10).fit(df_result,result_fold[j]['y_test']).best_estimator_
    return dict_model, meta_model

In [192]:
def make_blanding(X, y, dict_model, meta_model,param_model_GS, param_meta_model):
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3)
    result_model ={}
    res= None
    for i in dict_model.keys():
        dict_model[i] = GridSearchCV(dict_model[i], param_model_GS[i], cv=10).fit(X_train, y_train).best_estimator_
        result_model[i] = dict_model[i].predict(X_test)
        res = result_model[i] if type(res) == type(None) else res+result_model[i]
    print((res//len(dict_model.keys()))-y_test)    
    df_result = pd.DataFrame.from_dict(result_model)
    meta_model = GridSearchCV(meta_model, param_meta_model, cv=10).fit(df_result,y_test).best_estimator_
    return dict_model, meta_model

In [187]:
def predict_result(X, dict_model, meta_model):
    result_model ={}
    for i in dict_model.keys():
        result_model[i] = dict_model[i].predict(X)
    df_result = pd.DataFrame.from_dict(result_model)
    result = meta_model.predict(df_result) 
    return result

In [193]:
param_model_GS = {'Ridge':{'alpha':[0.1,0.5,1]}, 
              'Lasso':{'alpha':[0.1,0.5,1]}, 
              'KNeighborsRegressor':{'n_neighbors':[3,5,7,9]},
              'LinearSVR':{'C':[0.1,0.5,1], 'loss':['epsilon_insensitive', 'squared_epsilon_insensitive']},
              'DecisionTreeRegressor':{'max_depth':[3,5,7,9]},                                    
              'LinearRegression':{},
              'SGDRegressor':{'loss':['squared_loss', 'huber', 'epsilon_insensitive', 'squared_epsilon_insensitive'],
                              'penalty':['none', 'l2', 'l1','elasticnet'] 
                             }
             }



dict_model = {}
#dict_model['KNeighborsRegressor'] = KNeighborsRegressor()
dict_model['Ridge'] = Ridge()
dict_model['Lasso'] = Lasso()
dict_model['LinearSVR'] = LinearSVR()
#dict_model['SGDRegressor'] = SGDRegressor()
dict_model['DecisionTreeRegressor'] = DecisionTreeRegressor()
meta_model = LinearRegression()


stacking_dict_model, stacking_meta_model = make_stacking(X, y, dict_model, meta_model,param_model_GS, param_model_GS['LinearRegression'])
stacking_result = predict_result(X_out, stacking_dict_model, stacking_meta_model)

blending_dict_model, blending_meta_model = make_blanding(X, y, dict_model, meta_model,param_model_GS, param_model_GS['LinearRegression'])
blending_result = predict_result(X_out, blending_dict_model, blending_meta_model)






[276] 359
[103] 136
[-8.63860e+04 -1.10690e+04 -1.93590e+04  2.93940e+04 -2.00520e+04
  8.32190e+04  8.96220e+04  6.96280e+04  1.41017e+05  8.34810e+04
  9.86160e+04 -3.12540e+04 -6.82360e+04  8.49890e+04  1.05430e+04
  5.50550e+04 -5.97440e+04 -2.26450e+04 -4.16690e+04  1.97200e+04
  5.37440e+04 -4.22660e+04 -2.16810e+04 -6.71800e+04  1.05400e+05
 -6.50000e+03 -1.56450e+04 -5.10150e+04 -9.10000e+01  4.34260e+04
  6.10970e+04  4.80740e+04  7.63480e+04  1.12180e+04  5.15570e+04
 -8.64500e+03  8.63800e+03 -3.09630e+04  5.11400e+03 -4.80200e+03
 -1.41820e+04  6.19460e+04 -3.31860e+04 -1.85720e+04  2.57100e+04
  9.62180e+04  3.96770e+04 -3.24100e+04 -3.48860e+04 -1.82593e+05
 -4.36280e+04  4.30490e+04 -1.21017e+05 -3.07200e+03  1.38550e+04
  7.78700e+03  4.11770e+04  5.42410e+04  5.36110e+04 -4.14350e+04
 -2.99860e+04  1.06492e+05  1.19873e+05 -1.81690e+04  2.94580e+04
 -2.00000e+02 -6.42220e+04 -1.12060e+04 -6.14500e+03  1.30430e+04
  1.65400e+04  6.46470e+04  3.32910e+04  2.08820e+04  1.

In [174]:
blending_dict_model

{'Ridge': Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None,
    normalize=False, random_state=None, solver='auto', tol=0.001),
 'Lasso': Lasso(alpha=1, copy_X=True, fit_intercept=True, max_iter=1000,
    normalize=False, positive=False, precompute=False, random_state=None,
    selection='cyclic', tol=0.0001, warm_start=False),
 'LinearSVR': LinearSVR(C=0.1, dual=True, epsilon=0.0, fit_intercept=True,
      intercept_scaling=1.0, loss='squared_epsilon_insensitive',
      max_iter=1000, random_state=None, tol=0.0001, verbose=0),
 'DecisionTreeRegressor': DecisionTreeRegressor(criterion='mse', max_depth=9, max_features=None,
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')}

In [175]:
# сравнение моделей

In [176]:
from sklearn.metrics import mean_absolute_error


In [177]:
Rg  = GridSearchCV(Ridge(), param_model_GS['Ridge'], cv=10).fit(X,y).best_estimator_.predict(X_out)
Ls  = GridSearchCV(Lasso(), param_model_GS['Lasso'], cv=10).fit(X,y).best_estimator_.predict(X_out)
KNR = GridSearchCV(KNeighborsRegressor(), param_model_GS['KNeighborsRegressor'], cv=10).fit(X,y).best_estimator_.predict(X_out)
SVR = GridSearchCV(LinearSVR(), param_model_GS['LinearSVR'], cv=10).fit(X,y).best_estimator_.predict(X_out)
DTR = GridSearchCV(DecisionTreeRegressor(), param_model_GS['DecisionTreeRegressor'], cv=10).fit(X,y).best_estimator_.predict(X_out)
SGD = GridSearchCV(SGDRegressor(), param_model_GS['SGDRegressor'], cv=10).fit(X,y).best_estimator_.predict(X_out)

In [178]:
print('blending:', mean_absolute_error(blending_result,y_out))
print('stacking:', mean_absolute_error(stacking_result,y_out))
print('ridge   :', mean_absolute_error(Rg,y_out))
print('lasso   :', mean_absolute_error(Ls,y_out))
print('KNR     :', mean_absolute_error(KNR,y_out))
print('SVR     :', mean_absolute_error(SVR,y_out))
print('DTR     :', mean_absolute_error(DTR,y_out))
print('SGD     :', mean_absolute_error(SGD,y_out))

blending: 19160.2655174496
stacking: 26681.565484725652
ridge   : 20333.96873426156
lasso   : 20514.18909968701
KNR     : 27300.959665144597
SVR     : 20278.874574306912
DTR     : 25436.227266351794
SGD     : 182859.63386165525


In [179]:
from sklearn.metrics import r2_score

In [180]:
print('blending:', r2_score(blending_result,y_out))
print('stacking:', r2_score(stacking_result,y_out))
print('ridge   :', r2_score(Rg,y_out))
print('lasso   :', r2_score(Ls,y_out))
print('KNR     :', r2_score(KNR,y_out))
print('SVR     :', r2_score(SVR,y_out))
print('DTR     :', r2_score(DTR,y_out))
print('SGD     :', r2_score(SGD,y_out))

blending: 0.8338419115344864
stacking: 0.6545679396802275
ridge   : 0.8605925614126346
lasso   : 0.8604944146358685
KNR     : 0.2301631689014454
SVR     : 0.8596722055748542
DTR     : 0.7694465066045482
SGD     : -173333005543.7248


In [181]:
# 

In [85]:
z = [1,2,3] + [4]

In [86]:
z.remove(1)

In [190]:
z

[2, 3, 4]

In [52]:
2222-222+(22/2)-2

2009.0

In [162]:
res = None
type(res) == type(None)

True